# **Introduction**

This notebook included statistical fun facts that were left out from previous analysis. We put them together here for write-up. Will periodically update whenever we need extra information. We will try to add back these tiny fun facts to the original analysis for better flow, but if they stand alone fine, we will keep adding
them to this file.

In [ ]:
# !pip install geopandas folium matplotlib seaborn scipy
# !pip install esda
# !pip install splot
# # for google colab, had to reinstall some pacakges.

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import datetime as dt
import scipy

from sklearn.cluster import DBSCAN
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# visualization
import matplotlib.pyplot as plt
from matplotlib import colors as mcolors
import seaborn as sns
import folium
from folium.plugins import HeatMap
from folium import Marker
from folium.plugins import MarkerCluster
import plotly.express as px
import plotly.io as pio

# system and utility
import warnings
import os
import io
from IPython.display import IFrame
from google.colab import files

import matplotlib.pyplot as plt

# suppress warnings
warnings.filterwarnings('ignore')

# inline
%matplotlib inline

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_row', 1000)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# save cleaned evictions data to my drive
link1 = '/content/drive/My Drive/X999/evictions_pre_post_covid.csv'
link2 = '/content/drive/My Drive/X999/evictions_covid.csv'

In [ ]:
normal_df = pd.read_csv(link1)
covid_df = pd.read_csv(link2)

In [ ]:
brooklyn = normal_df[normal_df['borough'] == 'BROOKLYN']
manhattan = normal_df[normal_df['borough'] == 'MANHATTAN']
queens = normal_df[normal_df['borough'] == 'QUEENS']
bronx = normal_df[normal_df['borough'] == 'BRONX']
staten_island = normal_df[normal_df['borough'] == 'STATEN ISLAND']

In [ ]:
brooklyn.shape, manhattan.shape, queens.shape, bronx.shape, staten_island.shape

((21713, 22), (12060, 22), (13725, 22), (26701, 22), (2519, 22))

## **What are the average eviction counts per building per year in different boroughs**

In [ ]:
brooklyn.average_year_eviction_count.describe()

,average_year_eviction_count
count,21713.000000
mean,1.087118
std,3.356285
min,0.200000
25%,0.200000
50%,0.400000
75%,0.800000
max,35.600000


In [ ]:
manhattan.average_year_eviction_count.describe()

,average_year_eviction_count
count,12060.000000
mean,0.974411
std,1.517442
min,0.200000
25%,0.200000
50%,0.600000
75%,1.000000
max,13.800000


In [ ]:
staten_island.average_year_eviction_count.describe()

,average_year_eviction_count
count,2519.000000
mean,0.836284
std,1.197151
min,0.200000
25%,0.200000
50%,0.400000
75%,0.600000
max,5.600000


In [ ]:
queens.average_year_eviction_count.describe()

,average_year_eviction_count
count,13725.000000
mean,1.137647
std,2.522823
min,0.200000
25%,0.200000
50%,0.400000
75%,0.800000
max,20.600000


In [ ]:
bronx.average_year_eviction_count.describe()

,average_year_eviction_count
count,26701.000000
mean,1.576795
std,2.553449
min,0.200000
25%,0.400000
50%,1.000000
75%,1.800000
max,27.800000


In [ ]:
manhattan.average_year_eviction_count.describe()

,average_year_eviction_count
count,12060.000000
mean,0.974411
std,1.517442
min,0.200000
25%,0.200000
50%,0.600000
75%,1.000000
max,13.800000


In [ ]:
borough_dict = {'Brooklyn': brooklyn.average_year_eviction_count.mean(), 'Manhattan': manhattan.average_year_eviction_count.mean(),
                'Queens': queens.average_year_eviction_count.mean(), 'Bronx': bronx.average_year_eviction_count.mean(),
                'Staten Island': staten_island.average_year_eviction_count.mean()}

In [ ]:
sorted_list = sorted(borough_dict)

In [ ]:
sorted_list

['Bronx', 'Brooklyn', 'Manhattan', 'Queens', 'Staten Island']

### **In short, Bronx has the most average eviction counts, followed by brooklyn, manhattan, queens, and the Staten island.**

## **Check out the exact total records before 2025**

In [ ]:
link3 = '/content/drive/My Drive/X999/Evictions.csv'
raw_evictions = pd.read_csv(link3)

In [ ]:
raw_evictions.shape

(104457, 20)

In [ ]:
raw_evictions.columns

Index(['Court Index Number', 'Docket Number ', 'Eviction Address',
       'Eviction Apartment Number', 'Executed Date', 'Marshal First Name',
       'Marshal Last Name', 'Residential/Commercial', 'BOROUGH',
       'Eviction Postcode', 'Ejectment', 'Eviction/Legal Possession',
       'Latitude', 'Longitude', 'Community Board', 'Council District',
       'Census Tract', 'BIN', 'BBL', 'NTA'],
      dtype='object')

In [ ]:
raw_evictions['Executed Date'] = pd.to_datetime(raw_evictions['Executed Date'], errors='coerce')

In [ ]:
raw_evictions['year'] = raw_evictions['Executed Date'].dt.year
# need to explicitly declaire year

In [ ]:
raw_evictions.year.value_counts()

,count
Executed Date,
2017,22522
2018,21830
2019,18715
2024,16755
2023,13447
2022,5067
2020,3422
2025,2431
2021,268


In [ ]:
raw_evictions.columns

Index(['Court Index Number', 'Docket Number ', 'Eviction Address',
       'Eviction Apartment Number', 'Executed Date', 'Marshal First Name',
       'Marshal Last Name', 'Residential/Commercial', 'BOROUGH',
       'Eviction Postcode', 'Ejectment', 'Eviction/Legal Possession',
       'Latitude', 'Longitude', 'Community Board', 'Council District',
       'Census Tract', 'BIN', 'BBL', 'NTA', 'year'],
      dtype='object')

In [ ]:
raw_evictions = raw_evictions[raw_evictions['year'] != 2025]

In [ ]:
raw_evictions.shape
# used in write up

(102026, 21)

# **Clustering Patterns**

## **SVI Stuff**